In [ ]:
import spacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

In [ ]:
# # SpaCy uses nltk to handle wordnet
# !pip install nltk
# import nltk
# nltk.download('wordnet')

# !pip install spacy spacy-wordnet
# !python -m spacy download en_core_web_sm

In [ ]:
nlp('prices asd asd')

In [ ]:
# Load a spaCy model (supported languages are "es" and "en") 
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacy_wordnet", after='tagger')

In [ ]:
token = nlp('dog')[0]

In [ ]:
token._.wordnet.synsets()

In [ ]:
token._.wordnet.lemmas()

In [ ]:
token._.wordnet.wordnet_domains()

In [ ]:
# WordNet object links spaCy token with NLTK WordNet interface by giving access to
# synsets and lemmas 
token._.wordnet.synsets()
token._.wordnet.lemmas()

# And automatically add info about WordNet domains
token._.wordnet.wordnet_domains()